In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [ ]:
spark = (
    SparkSession
        .builder
        .master("local")
        .appName("Data Sources")
        .config("spark.jars", "jars/postgresql-42.7.2.jar")
        .config("spark.driver.memory", "16g")
        .getOrCreate()
)

## Драйвер

In [ ]:
driver = "org.postgresql.Driver"
url = "jdbc:postgresql://localhost:5432/spark"
user = "postgres"
password = "postgres"

## Чтение таблицы целиком

### Пример 1

In [ ]:
employees_df = spark.read \
    .format("jdbc") \
    .option("driver", driver) \
    .option("url", url) \
    .option("user", user) \
    .option("password", password) \
    .option("dbtable", "public.employees") \
    .load()

employees_df.count()

In [ ]:
employees_df.show(10)

### Пример 2

In [ ]:
DBPARAMS = {
    "user": user,
    "password": password,
    "driver": driver
}

In [ ]:
df = spark.read.jdbc(url=url, table="public.employees", properties=DBPARAMS)
df.count()

In [ ]:
df.show(10)

In [ ]:
df.rdd.getNumPartitions()

In [ ]:
df.agg(min(col("emp_no")), max(col("emp_no"))).show()

### Как распараллелить чтение?

In [ ]:
df101 = spark.read.jdbc(url=url, table="public.employees", properties=DBPARAMS, numPartitions=10)
df101.count()

In [ ]:
df101.rdd.getNumPartitions()

In [ ]:
df102 = spark.read.jdbc(url=url, table="public.employees", properties=DBPARAMS,
                        column="emp_no", lowerBound = 10010, upperBound = 499990, numPartitions=10)
df102.count()

In [ ]:
df102.rdd.getNumPartitions()

In [ ]:
df103 = spark.read \
    .format("jdbc") \
    .option("driver", driver) \
    .option("url", url) \
    .option("user", user) \
    .option("password", password) \
    .option("dbtable", "public.employees") \
    .option("partitionColumn", "emp_no") \
    .option("lowerBound", "10010") \
    .option("upperBound", "499990") \
    .option("numPartitions", "10") \
    .load()

df103.count()

In [ ]:
df103.rdd.getNumPartitions()

In [ ]:
employees_pruned = """(select e.first_name, e.last_name, e.hire_date from public.employees e where e.gender = 'F') as new_emp"""
df_pruned = spark.read.jdbc(url=url, table=employees_pruned, properties=DBPARAMS)
df_pruned.count()

In [ ]:
df_pruned.show(10)

## Предикаты

### Пример 1

In [ ]:
pred = ["gender = 'M'", "gender = 'F'"]

df_pred = spark.read.jdbc(url=url, table="public.employees", properties=DBPARAMS, predicates=pred)
df_pred.count()

In [ ]:
df_pred.rdd.getNumPartitions()

In [ ]:
pred1 = ["gender = 'F'"]

df_pred1 = spark.read.jdbc(url=url, table="public.employees", properties=DBPARAMS, predicates=pred1)
df_pred1.count()

In [ ]:
df_pred1.rdd.getNumPartitions()

In [ ]:
pred3 = ["gender = 'F'", "gender = 'M'", "gender = 'M'"]

df_pred3 = spark.read.jdbc(url=url, table="public.employees", properties=DBPARAMS, predicates=pred3)
df_pred3.count()

In [ ]:
df_pred3.rdd.getNumPartitions()

In [ ]:
df.groupBy(col("gender")).agg(count(col("emp_no"))).show()

In [ ]:
df_pred3.groupBy(col("gender")).agg(count(col("emp_no"))).show()

### Пример 2

In [ ]:
pred2 = ["emp_no > 20000 and emp_no <= 50000", "emp_no >= 50000 and emp_no <= 100000"]

df_pred2 = spark.read.jdbc(url=url, table="public.employees", properties=DBPARAMS, predicates=pred2)

df_pred2.count()

In [ ]:
df_pred2.show(10)

## Фильтрация

In [ ]:
q = """select * from public.employees where emp_no > 20000 and emp_no <= 50000"""

dfq = spark.read \
    .format("jdbc") \
    .option("driver", driver) \
    .option("url", url) \
    .option("user", user) \
    .option("password", password) \
    .option("query", q) \
    .load()

dfq.count()

In [ ]:
dfq.show(10)

## Соединения в базе

In [ ]:
qj = """select e.emp_no, birth_date, first_name, last_name, gender, hire_date, salary, from_date, to_date
from employees e join salaries s on e.emp_no = s.emp_no"""

dfj = spark.read \
    .format("jdbc") \
    .option("driver", driver) \
    .option("url", url) \
    .option("user", user) \
    .option("password", password) \
    .option("query", qj) \
    .load()

dfj.count()

In [ ]:
dfj.show()

## Запись в таблицу

In [ ]:
employees_df.show(10)

In [ ]:
salaries_df = spark.read \
    .format("jdbc") \
    .option("driver", driver) \
    .option("url", url) \
    .option("user", user) \
    .option("password", password) \
    .option("dbtable", "public.salaries") \
    .load()

salaries_df.count()

In [ ]:
salaries_df.show(10)

In [ ]:
employees_salaries_df = employees_df.join(salaries_df.groupBy(col("emp_no")).agg(max(col("salary")).alias("max_salary")), "emp_no")

employees_salaries_df.show()

In [ ]:
employees_salaries_df.write \
    .mode("overwrite") \
    .format("jdbc") \
    .option("driver", driver) \
    .option("url", url) \
    .option("user", user) \
    .option("password", password) \
    .option("dbtable", "public.employees_salaries") \
    .option("truncate", "true") \
    .save()